In [ ]:
import pandas as pd
xml = pd.read_xml("xml/lotto1956.xml")
print(xml)

In [ ]:
import pandas as pd

def transformer(div):
    d = []
    for item in div['div'][0]:
        if 'time' in item:
            d.append({
                'time': item['time'],
                'draw1': item['div'][1],
                'draw2': item['div'][2], 
                'draw3': item['div'][3], 
                'draw4': item['div'][4], 
                'draw5': item['div'][5],
                'draw6': item['div'][6],
                'draw7': item['div'][7],
            })
    return d

def convertColumnsInOneColumn(dataFrame):
    array = []
    for column in [str(x) for x in list(range(1,7))]:
        array.append(dataFrame["draw" + column])
    return array

def forAllFiles(fromYear, toYear):
    tmp = []
    for year in [str(x) for x in list(range(fromYear, toYear))]:
        div = pd.read_json("json/lotto" + year + ".json")
        transformed = pd.DataFrame(transformer(div))
        tmp.append(transformed)
    return tmp

overall = pd.concat(forAllFiles(1955, 2022))    
pd.concat(convertColumnsInOneColumn(overall)).value_counts()


In [ ]:
div1955 = pd.read_json("json/lotto1955.json")
div1956 = pd.read_json("json/lotto1956.json")
div = div1955.append(div1956)
div1956['div'][0]